In [1]:
# 1. Install required packages
!pip install pandas
!pip install folium


In [3]:
# 2. Import necessary libraries
import pandas as pd
import folium


In [5]:
# 3. Load your local CSV file
# Replace 'your_file.csv' with the actual filename if it's different
launch_sites_df = pd.read_csv('spacex_launch_geo.csv')

# View the first few rows to confirm it loaded properly
launch_sites_df.head()


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [7]:
# 4. Create a base map centered at NASA Johnson Space Center
site_map = folium.Map(location=[29.559684888503615, -95.0830971930759], zoom_start=5)

# 5. Add markers for each launch site in the DataFrame
for index, row in launch_sites_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],
        icon=folium.Icon(color='red', icon='rocket')
    ).add_to(site_map)

# Display the map
site_map


In [9]:
from folium.plugins import MarkerCluster


In [13]:
launch_sites_df['marker_color'] = launch_sites_df['class'].apply(lambda x: 'green' if x == 1 else 'red')


In [15]:
from folium.plugins import MarkerCluster

marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

for index, record in launch_sites_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=record['Launch Site'],
        icon=folium.Icon(color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map


In [5]:
from math import sin, cos, sqrt, atan2, radians
from folium.plugins import MousePosition
import folium
# 3. Define the Haversine Distance Function
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # approximate radius of earth in km
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# 4. Create or load your folium map (already created from previous steps)
# Assume 'site_map' already exists centered at NASA Johnson Space Center
# If not, create one:
site_map = folium.Map(location=[29.559684888503615, -95.0830971930759], zoom_start=5)

# 5. Add Mouse Position plugin to map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' | ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# --- Start adding distances between launch site and nearby features ---

# Example: Define Launch Site Coordinates (Example: CCAFS LC-40)
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# 6. Manually identify nearby features (hover over the map and get coordinates)

# Coastline coordinates
coastline_lat = 28.56342
coastline_lon = -80.5678

# Railway coordinates
railway_lat = 28.5721
railway_lon = -80.5858

# Highway coordinates
highway_lat = 28.5643
highway_lon = -80.5725

# Nearest city coordinates (example Melbourne)
city_lat = 28.0836
city_lon = -80.6081

# 7. Calculate Distances
distance_to_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_to_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)
distance_to_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)
distance_to_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# 8. Add Markers with Distance Labels

# Coastline Marker
coastline_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=folium.DivIcon(html=f"""<div style="font-size: 12px; color: blue;">{distance_to_coastline:.2f} km</div>""")
)
site_map.add_child(coastline_marker)

# Railway Marker
railway_marker = folium.Marker(
    [railway_lat, railway_lon],
    icon=folium.DivIcon(html=f"""<div style="font-size: 12px; color: green;">{distance_to_railway:.2f} km</div>""")
)
site_map.add_child(railway_marker)

# Highway Marker
highway_marker = folium.Marker(
    [highway_lat, highway_lon],
    icon=folium.DivIcon(html=f"""<div style="font-size: 12px; color: orange;">{distance_to_highway:.2f} km</div>""")
)
site_map.add_child(highway_marker)

# City Marker
city_marker = folium.Marker(
    [city_lat, city_lon],
    icon=folium.DivIcon(html=f"""<div style="font-size: 12px; color: purple;">{distance_to_city:.2f} km</div>""")
)
site_map.add_child(city_marker)

# 9. Draw Polylines Connecting Launch Site to Each Feature

# Coastline Line
folium.PolyLine(locations=[(launch_site_lat, launch_site_lon), (coastline_lat, coastline_lon)], color='blue', weight=2).add_to(site_map)

# Railway Line
folium.PolyLine(locations=[(launch_site_lat, launch_site_lon), (railway_lat, railway_lon)], color='green', weight=2).add_to(site_map)

# Highway Line
folium.PolyLine(locations=[(launch_site_lat, launch_site_lon), (highway_lat, highway_lon)], color='orange', weight=2).add_to(site_map)

# City Line
folium.PolyLine(locations=[(launch_site_lat, launch_site_lon), (city_lat, city_lon)], color='purple', weight=2).add_to(site_map)

# 10. Show final map
site_map